**Epochs**

**Batch size**

For data synthetisation I will be using the [SDV - Synthetic Data Vault](https://docs.sdv.dev/sdv/) library, that includes the most known data synthetisation methods.

In [ ]:
# Import the libraries
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold

import recordlinkage